# Trabajo Final - Criptografía Práctica y Blockchain

|    Nombre   |  Apellido  |     Padrón      |
|:-----------:|:----------:|:------------:|
|    Tomás     |   Rodriguez Dala    |    |
|    Mateo    |   Capón Blanquer    |  104258  |

## Enunciado
El objetivo de esta práctica es poner en conjunto varios de los temas vistos en la asignatura, a partir del uso de una biblioteca para pruebas de conocimiento nulo. En este caso, se utilizarán las bibliotecas https://github.com/lambdaclass/lambdaworks_cairo_prover/ y https://github.com/lambdaclass/lambdaworks/

Se proponen las siguientes consignas:
1. Crear el AIR para un cálculo simple (aplicación repetitiva de una función polinómica) y,
utilizando el prover, generar una prueba de validez computacional para el problema
seleccionado.
2. Desarrollar un programa que permita un cálculo utilizando el lenguaje CAIRO (sin utilizar
built-ins), generar una prueba de validez computacional y verificarla mediante el
programa verificador.
3. Explicar el diagrama de flujo para generar y verificar pruebas utilizando STARKs.
4. Analizar el efecto que tiene la cantidad de chequeos (queries) y blowup factor sobre el
tiempo de generación de la prueba y la memoria que ocupa la prueba.
5. Explicar los parámetros de los que depende la seguridad de la prueba ¿Cuáles son las
consecuencias de utilizar cuerpos más pequeños que el original?
6. ¿De qué manera se podría disminuir la memoria requerida por la prueba?


## Imports

In [2]:
:dep lambdaworks-stark = { path = "." }
:dep lambdaworks-math = { git = "https://github.com/lambdaclass/lambdaworks", rev = "a17b951" }
:dep lambdaworks-crypto = { git = "https://github.com/lambdaclass/lambdaworks", rev = "a17b951" }
:dep num-integer

In [3]:
use std::ops::Range;

use lambdaworks_math::field::fields::{
    fft_friendly::stark_252_prime_field::Stark252PrimeField as F,
    u64_prime_field::{F17, FE17},
};
use lambdaworks_stark::{
    cairo::{
        air::{
            generate_cairo_proof, verify_cairo_proof, MemorySegment, MemorySegmentMap,
            PublicInputs, FRAME_DST_ADDR, FRAME_OP0_ADDR, FRAME_OP1_ADDR, FRAME_PC,
        },
        cairo_layout::CairoLayout,
        execution_trace::build_main_trace,
        runner::run::{
            cairo0_program_path, cairo1_program_path, generate_prover_args, run_program,
            CairoVersion,
        },
    },
    starks::{
        example::{
            dummy_air::{self, DummyAIR},
            fibonacci_2_columns::{self, Fibonacci2ColsAIR},
            fibonacci_rap::{fibonacci_rap_trace, FibonacciRAP, FibonacciRAPPublicInputs},
            quadratic_air::{self, QuadraticAIR, QuadraticPublicInputs},
            simple_fibonacci::{self, FibonacciAIR, FibonacciPublicInputs},
        },
        proof::options::{ProofOptions, SecurityLevel},
        proof::stark::StarkProof,
        prover::prove,
        trace::TraceTable,
        verifier::verify,
    },
    FE,
};
use lambdaworks_math::traits::Serializable
use lambdaworks_crypto::fiat_shamir::transcript::Transcript;
use lambdaworks_math::field::{element::FieldElement, traits::IsFFTField};

use lambdaworks_stark::starks::{
    constraints::boundary::{BoundaryConstraint, BoundaryConstraints},
    context::AirContext,
    frame::Frame,
    proof::options::ProofOptions,
    trace::TraceTable,
    traits::AIR,
};
use std::sync::atomic::{AtomicUsize, AtomicU64, AtomicIsize, Ordering::SeqCst};
use std::time::{Instant, Duration};
use num_integer::binomial;

## Ejercicio 1

Crear el AIR para un cálculo simple (aplicación repetitiva de una función polinómica) y,
utilizando el prover, generar una prueba de validez computacional para el problema
seleccionado.

### Resolución

Utilizamos la siguiente relación de recurrencia:

$$
P_n = 2 \cdot P_{n-1} + P_{n - 2},
$$
junto con las siguientes condiciones de contorno:

$$
P_0 = 0,
P_1 = 1
$$

A continuación explicamos lo que debimos implementar del AIR.

En principio para hacer un cálculo simple, de una sola columna, se eligen los siguientes valores en el contexto:

1. `transition_degrees = vec![1]`, porque la restricción es de grado 1.
2. `transition_offsets = vec![0, 1, 2]`, porque se necesitan las dos anteriores filas para calcular el valor actual.
3. `num_transition_constraints = 1`, porque hay solo una restricción de transición que define a la relación.
4. `transition_exemptions = vec![2]`, porque se deben exceptuar dos restricciones de transicion (por como está definida la restricción de transición).


Luego, para computar la transición, simplemente despejamos la relación de recurrencia a un lado del igual. Y por último, las restricciones de contorno son las indicadas anteriormente. Suponiendo que lo hacemos con una sola columna, las restricciones son simples de definir.

A continuación mostramos una implementación del AIR, genérico para aceptar n columnas y distintos valores de los coeficientes de la restricción de transición. Como no quisimos modificar el código del trait AIR, decidimos usar variables globales como solución rápida y fácil para la configuración del trait. De este modo, podemos modificar las variables globales N_COLS, FIRST_FACTOR y SECOND_FACTOR, para poder usar al AIR de distintos modos.

#### Variables para hacer el AIR un poco más genérico

In [62]:
// Cantidad de columnas que se van a usar en el AIR.
static N_COLS: AtomicUsize = AtomicUsize::new(1);

// Factores en la restriccion de transicion. P_n = FF * P_{n -1} + SF * P_{n -1}
static FIRST_FACTOR: AtomicU64 = AtomicU64::new(2);
static SECOND_FACTOR: AtomicIsize = AtomicIsize::new(1);

pub fn get_factors() -> (u64, isize) {
    (FIRST_FACTOR.load(SeqCst), SECOND_FACTOR.load(SeqCst))
}

pub fn change_n_cols(new_value: usize) -> usize {
    N_COLS.store(new_value, SeqCst);
    new_value
}

pub fn get_n_cols() -> usize {
    N_COLS.load(SeqCst)
}

// Esta funcion la utilizamos en pruebas nuestras para ver como se comportan los algoritmos
// si se elige una subsecuencia de la relacion. 
// Por ejemplo, en vez de tener la subsecuencia 1,2,3,4,5; con un jumpFactor de 2,
// se tendrian solo los numeros 1,3,5,...
// Aplicamos una idea similar pero para la relacion P_n = 2 P_{n-1} + P_{n-2}
pub fn update_factors(jump_factor: u32) -> (FieldElement<F>, FieldElement<F>) {
    let mut first_factor = 0;
    // Las raices del polinomio caracteristo son 1 + srqrt(2) y 1 - srqrt(2).
    // Por lo tanto, la relacion se puede expresar como 
    // P_n = a (1 + srqrt(2))^n + b (1 - srqrt(2))^n.
    // Si buscamos una subrelacion -una relacion que saltee algunos elementos de
    // la relacion para llegar a un resultado en menos pasos-, podemos pensar en un 
    // P_n*c = a (1 + srqrt(2))^nc + b (1 - srqrt(2))^nc. (c es un factor de escala o jump_factor)
    // Esta ecuacion tiene un polinomio caracteristico con raices en 
    // (1 + srqrt(2))^c y (1 - srqrt(2))^c, y su polinomio caracteristico se puede escribir como
    // x^2 - [(1 + srqrt(2))^c + (1 - srqrt(2))^c] x + (-1)^c.
    // De esto se obtienen el first_factor y second_factor. Para no tener que hacer (1 - srqrt(2))^c,
    // desarrollamos el coeficiente para hacer el calculo con enteros siempre (lo que se hace a continuacion).
    for k in 0..(jump_factor + 1) {
        if k % 2 == 0 {
            first_factor += binomial(jump_factor, k) * u32::pow(2, k / 2);
        }
    }
    first_factor *= 2;
    let second_factor = if jump_factor % 2 == 1 {1} else {-1};
    FIRST_FACTOR.store(first_factor as u64, SeqCst);
    SECOND_FACTOR.store(second_factor, SeqCst);
    println!("La restriccion de transicion es: P(n) = {} * P(n-1) + ({}) * P(n-2)", first_factor, second_factor);
    let c0 = FieldElement::<F>::from(first_factor as u64);
    let c1 = if second_factor == -1 {
        FieldElement::<F>::zero() - FieldElement::<F>::one()
    } else {// Vale 1.
        FieldElement::<F>::one()
    };
    (c0, c1)
}

#### Implementación del AIR

In [63]:
#[derive(Clone)]
pub struct PellAIR<F>
where
    F: IsFFTField,
{
    context: AirContext,
    trace_length: usize,
    pub_inputs: PellPublicInputs<F>,
    n_columns: usize,
    first_factor: FieldElement<F>,
    second_factor: FieldElement<F>,
}

#[derive(Clone, Debug)]
pub struct PellPublicInputs<F>
where
    F: IsFFTField,
{
    pub a0: FieldElement<F>,
    pub a1: FieldElement<F>,
}

impl<F> AIR for PellAIR<F>
where
    F: IsFFTField,
{
    type Field = F;
    type RAPChallenges = ();
    type PublicInputs = PellPublicInputs<Self::Field>;

    fn new(
        trace_length: usize,
        pub_inputs: &Self::PublicInputs,
        proof_options: &ProofOptions,
    ) -> Self {
        let n_columns = get_n_cols();
        // Solo para el caso de una columna se necesitan las dos filas anteriores.
        // Para mas de una columna, se necesitan los valores en la ultima fila y en 
        // la actual.
        let transition_offsets = if n_columns > 1 {vec![0, 1]} else {vec![0, 1, 2]};
        
        // en total siempre se deben exceptuar dos restricciones de transicion. En el caso de 
        // una sola columna, son los dos primeros valores por ser condiciones de contorno. Si 
        // se tiene mas de una columna, estas restricciones estaran en la primera y la segunda
        // columna. Los demas, sí deben ser evaluados.
        let transition_exemptions = if n_columns == 1 {
            vec![2] 
        } else {
            vec![1, 1].into_iter().chain(std::iter::repeat(0).take(n_columns - 2)).collect()
        }; 
        
        let context = AirContext {
            proof_options: proof_options.clone(),
            trace_columns: n_columns,
            transition_degrees: vec![1; n_columns], // usamos la misma transicion en cada columna. 
            transition_offsets: transition_offsets,
            num_transition_constraints: n_columns,
            transition_exemptions: transition_exemptions,
            num_transition_exemptions: 1,
        };
        // Traemos las variables globales aca. 
        // Sabemos que no es muy bonito a nivel programacion, pero queriamos analizar distintas metricas
        // para distintos factores en las transiciones y nos parecio lo mas sencillo.
        let (c0, c1) = get_factors(); 
        let second_factor = if c1 < 0 {
            FieldElement::<F>::zero() - FieldElement::<F>::from((c1 * (-1)) as u64)
        } else {// Es usize porque c1 > 0.
            FieldElement::<F>::from(c1 as u64)
        };
        Self {
            pub_inputs: pub_inputs.clone(),
            context,
            trace_length,
            n_columns,
            first_factor: FieldElement::<F>::from(c0),
            second_factor,
        }
    }

    fn composition_poly_degree_bound(&self) -> usize {
        self.trace_length()
    }

    fn build_auxiliary_trace(
        &self,
        _main_trace: &TraceTable<Self::Field>,
        _rap_challenges: &Self::RAPChallenges,
    ) -> TraceTable<Self::Field> {
        TraceTable::empty()
    }

    fn build_rap_challenges<T: Transcript>(&self, _transcript: &mut T) -> Self::RAPChallenges {}

    fn compute_transition(
        &self,
        frame: &Frame<Self::Field>,
        _rap_challenges: &Self::RAPChallenges,
    ) -> Vec<FieldElement<Self::Field>> {
        let first_row = frame.get_row(0);
        let second_row = frame.get_row(1);
        if self.n_columns > 1 {
            let mut transitions = vec![];
            // La primera restriccion es tal que s_{0, i+1} = FF * s_{ncols - 1, i} + SF * s_{ncols - 2, i}.
            transitions.push(second_row[0].clone() - 
                 self.first_factor.clone() * first_row[self.n_columns - 1].clone() - 
                 self.second_factor.clone() * first_row[(2 * self.n_columns - 2) % self.n_columns].clone());
            
            // La segunda restriccion tambien usa la fila anterior siempre. 
            transitions.push(second_row[1].clone() - 
                self.first_factor.clone() * second_row[0].clone() - 
                self.second_factor.clone() * first_row[self.n_columns - 1].clone());
            
            // Las demas transiciones son sobre la misma fila pero de las anteriores dos columnas.
            for i in 2..self.n_columns {
                transitions.push(second_row[i].clone() - 
                    self.first_factor.clone() * second_row[i-1].clone() - 
                    self.second_factor.clone() * second_row[i-2].clone());
            }
            return transitions;
        }
        // Cuando hay una sola columna necesitamos la fila anterior para poder calcular la restriccion.
        let third_row = frame.get_row(2);

        vec![third_row[0].clone() - 
            self.first_factor.clone() * second_row[0].clone() - 
            self.second_factor.clone() * first_row[0].clone()]
    }

    fn boundary_constraints(
        &self,
        _rap_challenges: &Self::RAPChallenges,
    ) -> BoundaryConstraints<Self::Field> {
        let a0 = BoundaryConstraint::new(0, 0, self.pub_inputs.a0.clone());
        // si se hace con mas de una columna, la segunda restriccion es el primer
        // valor de la segunda columna.
        let a1 = if self.n_columns > 1 { 
            BoundaryConstraint::new(1, 0, self.pub_inputs.a1.clone())
        } else { 
            BoundaryConstraint::new(0, 1, self.pub_inputs.a1.clone())
        };

        BoundaryConstraints::from_constraints(vec![a0, a1])
    }

    fn number_auxiliary_rap_columns(&self) -> usize {
        0
    }

    fn context(&self) -> &AirContext {
        &self.context
    }

    fn trace_length(&self) -> usize {
        self.trace_length
    }

    fn pub_inputs(&self) -> &Self::PublicInputs {
        &self.pub_inputs
    }
}

#### Cálculo de la Traza

In [65]:
pub fn pell_trace<F: IsFFTField>(
    initial_values: [FieldElement<F>; 2],
    trace_length: usize,
    n_columns: usize,
    first_factor: FieldElement<F>,
    second_factor: FieldElement<F>,
) -> TraceTable<F> {
    let mut ret: Vec<Vec<FieldElement<F>>> = vec![vec![]; n_columns];
    ret[0].push(initial_values[0].clone());
    
    // si hay mas de una columna, es el primer elemento de la segunda columna.
    // Sino, en la unica columna que hay 
    ret[1 % n_columns].push(initial_values[1].clone());
    let mut a_n_1 = initial_values[1].clone();
    let mut a_n_2 = initial_values[0].clone();

    for i in 2..(trace_length * n_columns) {
        let res = first_factor.clone() * a_n_1.clone() + second_factor.clone() * a_n_2.clone();
        ret[i % n_columns].push(res.clone());
        a_n_2 = a_n_1;
        a_n_1 = res;
    }
    TraceTable::new_from_cols(&ret)
}

#### Prueba de Validez

In [66]:
fn run_test_prove(
    n_cols: usize,
    trace_length: usize,
    first_factor: FieldElement<F>,
    second_factor: FieldElement<F>,
    pub_inputs: PellPublicInputs<F>,
    proof_options: ProofOptions,
    // este parametro lo usamos a modo de validacion, pero no es necesario en la prueba de validez.
    expected_result: FieldElement<F>, 
) -> (Duration, Duration, Duration, usize) {
    let mut now = Instant::now();
    let trace = pell_trace([pub_inputs.a0.clone(), pub_inputs.a1.clone()], 
                            trace_length, n_cols, first_factor, second_factor);

    let trace_time = now.elapsed();
    
    // esta validacion es de debug, la hacemos nosotros a modo de prueba para cuando 
    // usamos mas columnas y distintos factores.
    assert_eq!(expected_result, trace.get(trace_length - 1, n_cols - 1));
    
    now = Instant::now();
    let proof = prove::<F, PellAIR<F>>(&trace, &pub_inputs, &proof_options).unwrap();
    let proof_time = now.elapsed();
    let proof_size = proof.serialize().len();
    now = Instant::now();
    assert!(verify::<F, PellAIR<F>>(
        &proof,
        &pub_inputs,
        &proof_options
    ));
    let verify_time = now.elapsed();
    
    (trace_time, proof_time, verify_time, proof_size)
}

#### Ejecución de la prueba

In [67]:
let original_pub_inputs = PellPublicInputs {
    a0: FE::zero(),
    a1: FE::one(),
};
let n_cols = 1;
// calculamamos el P_1023.
let trace_length = 1024; 

// es nuestro valor de jumps para buscar subsecuencias. 
// Al ponerlo en 1, usamos la secuencia tal como la definimos arriba: P_n = 2 P_{n-1} + P_{n-2}.
let n_jumps = 1; 
let (original_c0, original_c1) = update_factors(n_jumps);

// Esta validacion la hacemos nosotros a modo de prueba para cuando usamos mas columnas y distintos factores.
// No es parte del proceso de la prueba de validez.
// Es para validar que todas las opciones que se nos ocurren
// para hacer el mismo calculo obtienen el mismo resultado.
let trace = pell_trace([original_pub_inputs.a0.clone(), 
                        original_pub_inputs.a1.clone()],
                        trace_length, n_cols, original_c0.clone(), original_c1.clone());
let expected_result = trace.get(trace_length - 1, n_cols - 1);

let proof_options = ProofOptions::default_test_options();

let (trace_time, proof_time, verify_time, proof_size) = 
     run_test_prove(n_cols, trace_length, original_c0.clone(), original_c1.clone(),
                    original_pub_inputs.clone(), proof_options.clone(), expected_result.clone());

println!("Tiempo de ejecución del calculo de la traza: {}µs", trace_time.as_micros());
println!("Tiempo de ejecución de la prueba: {}ms", proof_time.as_millis());
println!("Tiempo de ejecución de la verificación: {}ms", verify_time.as_millis());
println!("Memoria ocupada por la prueba: {}b", proof_size);

La restriccion de transicion es: P(n) = 2 * P(n-1) + (1) * P(n-2)
Tiempo de ejecución del calculo de la traza: 81µs
Tiempo de ejecución de la prueba: 44ms
Tiempo de ejecución de la verificación: 2ms
Memoria ocupada por la prueba: 20344b


## Ejercicio 2
Desarrollar un programa que permita un cálculo utilizando el lenguaje CAIRO (sin utilizar built-ins), generar una prueba de validez computacional y verificarla mediante el programa verificador.

### Resolución


## Ejercicio 3
Explicar el diagrama de flujo para generar y verificar pruebas utilizando STARKs.

### Resolución

#### Conceptos Generales
El protocolo se separa a grandes rasgos en dos entidades: un cliente o verifier por un lado y un servidor o prover, por el otro. El prover realiza un cálculo y publica el resultado del mismo. Utilizando STARK, se puede probar con una probabilidad sumamente alta que el resultado obtenido es el correcto. Esto se hace sin la necesidad de publicar todo el proceso de cómputo. Este concepto es conocido como Zero Knowledge Proof.

Una prueba STARK se realiza sobre la ejecución de un algoritmo que se puede representar como un conjunto de pasos secuenciales que cumple con determinadas restricciones que se pueden representar utilizando polinomios.


#### La prueba
El proceso de prueba consiste principalemente de 3 etapas (o rondas en la biblioteca) que permiten generar un esquema de compromiso al resultado obtenido.

##### Compromiso a la traza calculada

En la primera etapa, el Prover se compromete a la traza de ejecución del algoritmo. La traza de ejecución es el resultado de cada uno de estos pasos que se debieron realizar para llegar al resultado. Una vez que se realiza el cálculo sobre una función $f(g^n) = a_n$, se obtiene una traza con los siguientes resultados.

&nbsp;|&nbsp;&nbsp;&nbsp;x&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;f&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
---:|:---:|:---:|
&nbsp;|$g^0$|&nbsp;&nbsp;$a_0$&nbsp;&nbsp;|
&nbsp;|$g^1$|&nbsp;&nbsp;$a_1$&nbsp;&nbsp;|
&nbsp;|$g^2$|&nbsp;&nbsp;$a_2$&nbsp;&nbsp;|
&nbsp;|$g^3$|&nbsp;&nbsp;$a_3$&nbsp;&nbsp;|
&nbsp;|...|&nbsp;&nbsp;...&nbsp;&nbsp;|
&nbsp;|$g^n$|&nbsp;&nbsp;$a_n$&nbsp;&nbsp;|

Utilizando estos valores, se caclula el polinomio interpolador de grado a lo sumo n. Esto se puede hacer por ejemplo, con el algoritmo de Lagrange. Habiendo definido un dominio de evaluación lo suficientemente grande, se evalua el polinomio en todos esos puntos y se realiza el Merkle Tree sobre esos resultados. Se almacena el arbol completo y se publica la raiz del Merkle Tree.

##### Compromiso al polinomio de Composición (CP)
En la segunda etapa, el prover se compromete a que se cumplen las restricciones previamente definidas en el AIR. Existen dos tipos de restricciones. Por un lado, las restricciones de contorno, que son aquellas evaluaciones del polinomio que deben cumplirse. Por ejemplo, $a_0 = 0$ es una restricción de contorno. Por el otro, las restricciones de transición son aquellas relaciones que se deben cumplir entre los elementos. Estos son definidos por el polinomio elegido.

La representación de las restricciones en función de la sucesión se puede asociar a una representación en el mundo de los polinomios. Por ejemplo, si el prover asegura que el valor de $a_i$ es $c$, esto es equivalente a decir que $f(g^i) = c$, con $g$ un generador de grupo G. Por lo tanto, la restricción se puede reescribir asegurando que $f$ - c tiene una raiz en $g^i$. Esto se cumple si y solo si, la siguiente función racional es un polinomio en todos sus puntos (menos en $g^i$).

$$p_j(x) = \frac{f(x) - c}{x - g^i}$$

En esta etapa, se calculan los polinomios $p_j(x)$ que representan a estas funciones racionales. Los mismos se comportan como polinomios, si se cumplen las restricciones que representan.

Una vez calculados estos m polinomios, se genera una última función, $CP(X)$ (*Composition Polynomial*), la cual vale:

$$CP(X) = \sum\limits_{i=0}^{m} \alpha_i \cdot p_i(x) $$

donde los $\alpha_i$ son números aleatoreos. Con alta probabilidad, si los $p_i$ son polinomios, entonces $CP(X)$ también lo es. 

Vale aclarar que la biblioteca utiliza otra estrategia para calcular el polinomio de Composición. Genera un polinomio B que se corresponde a las restricciones de contorno, y un polinomio C, con las de transición y define a CP como 

$$CP(X) = H(X) =  B \cdot (\alpha_1 \cdot x^{D - deg(B)} + \beta_1) + C \cdot (\alpha_2 \cdot x^{D - deg(C)} + \beta_2) $$

Se genera un esquema de Commitment con el polinomio $CP(X)$, publicando los coeficientes $\alpha_i$ y la raiz del Merkle Tree de su evaluación en el dominio.


##### Compromiso de que CP es un polinomio

Para convencer al verificador que $CP(X)$ es un polinomio, se busca probar que $CP(X)$ está muy "cerca" de un polinomio de grado bajo. Aqui es cuando se utiliza el protocolo FRI. A grandes rasgos consiste en reducir en la mitad el orden del polinomio CP iterativamente con transformaciones pertinentes hasta que quede un polinomio constante. Estos polinomios también tienen coeficientes aleatoreas $\beta_i$, para cada polinomio reducido $PC_i(X)$. Se realiza el Merkle Tree sobre cada Layer del protocolo y se publican todas las raices, y el valor final del polinomio constante. 


Antes de realizar la prueba, la biblioteca samplea ciertos valores del dominio de evaluación para que sean enviados al verificador. Los evalua en $f$ y en $H$ y los publica. De este modo, el verificador no tendrá que mantener una comunicación con el Prover para verificar que lo publicado es correcto.

Además, la biblioteca utiliza otra estrategia para probar que CP es un polinomio. En lugar de ejecutar el protocolo FRI sobre CP, lo hace sobre un polinomio un poco más extenso que incluye restricciones sobre los valores de la traza en los valores sampleados.


##### Valores publicados por el Prover
Hasta aquí el Prover publica:

1. root del Merkle Tree de la traza $f$
2. root del Merkle Tree de $CP$
3. root de los Merkle Tree de las evaluaciones de los polinomios $CP_i$
4. Valor del polinomio constante en FRI.
5. Valores de las constantes aleatorias alphas que son los coeficientes de $CP$.
6. Valores de las constantes aleatorias betas que son los coeficientes de $CP_i$.

#### La verificación
En el proceso de verificación, el verifier debe corroborar que el resultado publicado por el Prover es verdadero. Si la comunicación fuera sincrónica, elige un valor aleatorio $x$ dentro del dominio de evalución (o muchos si quieren hacer muchas queries), y le solicita al Prover el resultado de la evaluación de f en ese valor, y en los k valores siguientes dependiendo de la definición del polinomio. Además, el Prover provee el authentication Path que se debe seguir para llegar al valor del hash de la raiz, previamente publicado por el Prover. De este modo, el verifier se asegura que la evaluación de f en ese punto es verdaderamente el valor entregado por el Prover.

Luego se debe hacer el decommit sobre $CP$. Para esto, el Verifier calcula el valor de $CP(x)$ conociendo el valor de $f(x)$ y de los $\alpha_i$, y solicita al Prover el authentication Path para $x$, con el que verifica.

Por último, debe verificar que CP es un polinomio de grado bajo. Para esto, solicita también $CP(x^2)$, con el que puede calcular el valor de $CP_1(x)$, utilizando los $\beta_i$ recibidos anteriormente. Este valor lo verifica solicitando también el Authentication Path de $CP_1(x)$. Este proceso de verificación lo realiza iterativamente hasta que se llega al resultado del polinomio constante publicado por el Prover.


Vale destacar que esta comunicación no es exactamente asi en la biblioteca que utilizamos. El prover provee los valores sampleados y sus evaluaciones con anterioridad dentro de la prueba que publica.

## Ejercicio 4
Analizar el efecto que tiene la cantidad de chequeos (queries) y blowup factor sobre el tiempo de generación de la prueba y la memoria que ocupa la prueba.

### Resolución

El blowup factor (BF) representa al factor por el que se multiplica el largo de la traza para obtener el orden del dominio de evaluación. Las operaciones que más influyen en el tiempo de la generación de la prueba son:
1. Cálculo del polinomio interpolador.
2. Evaluación y commitment del polinomio de la traza sobre el dominio de evaluación.
3. Evaluación y commitment de cada $CP_i$ sobre el dominio de evaluación.

Las últimas dos dependen directamente del blowup factor, ya que el orden del dominio de evaluación vale blowup factor veces el largo de la traza.

Evaluar un polinomio BF veces más implica un aumento de BF veces sobre el tiempo de evaluación (asumiendo que todas las evaluaciones tardan lo mismo). Además, el commitment también implica un aumento de aproximadamente BF veces en el tiempo. Si se hace un commitment sobre un dominio de orden D, se crea un Merkle Tree de $2 \cdot D - 1$ nodos. Si el dominio aumenta en BF, el Merkle Tree tendrá $2 \cdot D \cdot BF - 1$ nodos. Por lo tanto se calculan BF hashes más, los cuales también deben ser almacenados, incrementando en BF veces este almacenamiento.

Para el caso de los $CP_i$, se puede pensar que duplicar el dominio de evaluación con respecto al largo de la traza (BF = 2) implica agregar un Layer más, con un dominio de evaluación dos veces más grande. Si el dominio es de orden D, se hacen aproximadamente 2D evaluaciones en total para los $CP_i$. Al duplicarlo, se hacen $2 \cdot D \cdot BF$ evaluaciones. La cantidad de hashes calculados también se duplica al agregar un Layer. 

Siendo que el Prover principalmente almacena los Merkle Trees, y la cantidad de nodos aumenta BF veces con respecto al BF, en resumen, tanto el almacenamiento temporal como espacial se multiplica con respecto al BF.

Esto lo mostramos en código en la siguiente celda.

In [68]:
fn run_many_test_prove(
    n_cols: usize,
    trace_length: usize,
    first_factor: FieldElement<F>,
    second_factor: FieldElement<F>,
    pub_inputs: PellPublicInputs<F>,
    proof_options: ProofOptions,
    // este parametro lo usamos a modo de validacion, pero no es necesario en la prueba de validez.
    expected_result: FieldElement<F>,
    n_loops: u32,
) -> (Duration, Duration, Duration, usize) {
    let (mut trace_time, mut proof_time, mut verify_time, mut proof_size) = 
        (Duration::ZERO, Duration::ZERO, Duration::ZERO, 0);  
    for i in 0..n_loops {
        let (t, p, v, s) = run_test_prove(n_cols, trace_length, first_factor.clone(), second_factor.clone(),
               pub_inputs.clone(), proof_options.clone(), expected_result.clone());
        trace_time += t; proof_time += p; verify_time += v; proof_size += s;
    }
    trace_time /= n_loops; proof_time /= n_loops; verify_time /= n_loops; proof_size /= n_loops as usize;
    (trace_time, proof_time, verify_time, proof_size)
}


In [70]:
let n_loops = 100;
let max_blowup = 5; // hasta 2^5 = 32

let mut proof_options = ProofOptions::default_test_options();
proof_options.blowup_factor = 1;

println!("| Blowup factor | Ej. Traza | Ej. Prueba | Ej. Verificacion | Tamaño prueba |");

for i in 0..max_blowup {
    proof_options.blowup_factor *= 2;
    let (trace_time, proof_time, verify_time, proof_size) = 
                     run_many_test_prove(n_cols, trace_length, 
                                         original_c0.clone(), original_c1.clone(),
                                         original_pub_inputs.clone(), proof_options.clone(), 
                                         expected_result.clone(), n_loops);
    println!("|{:>8}       | {:>5}µs   |{:>6}ms    | {:>8}ms       | {:>9}b    |",
          proof_options.blowup_factor, trace_time.as_micros(), 
          proof_time.as_millis(), verify_time.as_millis(), proof_size);
}

| Blowup factor | Ej. Traza | Ej. Prueba | Ej. Verificacion | Tamaño prueba |
|       2       |    66µs   |    19ms    |        1ms       |     18232b    |
|       4       |    62µs   |    35ms    |        2ms       |     20344b    |
|       8       |    62µs   |    68ms    |        4ms       |     22456b    |
|      16       |    66µs   |   143ms    |        8ms       |     24568b    |
|      32       |    63µs   |   282ms    |       18ms       |     26680b    |


()

In [71]:
proof_options.blowup_factor = 4;
proof_options.fri_number_of_queries = 1;
let n_type_queries_to_try = 8;

println!("| Number Queries | Ej. Traza | Ej. Prueba | Ej. Verificacion | Tamaño prueba |");
for i in 0..n_type_queries_to_try {
    proof_options.fri_number_of_queries *= 2;
    let (trace_time, proof_time, verify_time, proof_size) = 
                     run_many_test_prove(n_cols, trace_length, 
                                         original_c0.clone(), original_c1.clone(),
                                         original_pub_inputs.clone(), proof_options.clone(), 
                                         expected_result.clone(), n_loops);
    println!("|{:>9}       | {:>5}µs   |{:>6}ms    | {:>8}ms       | {:>9}b    |",
          proof_options.fri_number_of_queries, trace_time.as_micros(), 
          proof_time.as_millis(), verify_time.as_millis(), proof_size);
}

| Number Queries | Ej. Traza | Ej. Prueba | Ej. Verificacion | Tamaño prueba |
|        2       |    64µs   |    35ms    |        2ms       |     13784b    |
|        4       |    64µs   |    35ms    |        2ms       |     26904b    |
|        8       |    70µs   |    38ms    |        3ms       |     53144b    |
|       16       |    70µs   |    38ms    |        4ms       |    105624b    |
|       32       |    65µs   |    36ms    |        6ms       |    210584b    |
|       64       |    65µs   |    35ms    |       10ms       |    420504b    |
|      128       |    66µs   |    35ms    |       18ms       |    840344b    |
|      256       |    67µs   |    35ms    |       34ms       |   1680024b    |
|      512       |    71µs   |    36ms    |       67ms       |   3359384b    |
|     1024       |    84µs   |    40ms    |      136ms       |   6718104b    |


()

## Ejercicio 5
Explicar los parámetros de los que depende la seguridad de la prueba ¿Cuáles son las consecuencias de utilizar cuerpos más pequeños que el original?


## Ejercicio 6
¿De qué manera se podría disminuir la memoria requerida por la prueba?

### Resolución

Hay múltiples estrategias que se pueden utilizar para reducir la memoria de la prueba. Por un lado, se pueden agregar más columnas, y modificar las transiciones. De este modo, el largo de la traza es menor para un mismo problema. Esto implica, en terminos de memoria, almacenar menos nodos de los Merkle Trees, y tener menos capas en el protocolo de FRI. Esta estrategia la evaluamos en nuestro AIR utilizando el parámetro N_COLS.

Una solución que puede servir muchas veces es utilizar un algoritmo más eficiente, o que tenga una traza de orden menor, pero que haga el mismo cálculo que se desea probar. Un ejemplo muy claro de esto lo vimos en el trabajo práctico 2, donde achicabamos el orden de la traza al aplicar un paso más grande entre resultados consecutivos (elevar a la 8 en vez de elevar a la 2). Con este método se achican la cantidad de pasos a ejecutar, y por lo tanto la prueba será más eficiente tanto en términos temporales como en memoria.

Para nuestro caso, $P_n = 2 \cdot P_{n-1} + P_{n - 2}, P_0 = 0, P_1 = 1$, se tiene, por ejemplo, que la relación $Q_n = 6 \cdot Q_{n-1} - Q_{n - 2}, Q_0 = 1, Q_1 = 5$ cumple: $P_{2n+1} = Q_n$. Mostramos algunos valores en la siguiente tabla a modo de ejemplo.

|   n   |   m   | P(n) | Q(m) |
| :---: | :---: | :--: | :--: |
|   0   |   -   |  0   |  -   |
|   1   |   0   |  1   |  1   |
|   2   |   -   |  2   |  -   |
|   3   |   1   |  5   |  5   |
|   4   |   -   |  12  |  -   |
|   5   |   2   |  29  |  29  |
|   6   |   -   |  70  |  -   |
|   7   |   3   | 169  | 169  |

Estas relaciones son fáciles de encontrar en relaciones de recurrencia lineales como la nuestra. Para un algoritmo genérico en cairo, debemos buscar hacerlo lo más eficiente posible, asi se traduce en trazas más cortas.

Por último, si se reduce el blowup factor, se tendrá un dominio de evaluación menor, y por lo tanto, la memoria que ocupa la prueba disminuirá.


Todas estas estrategias deben evaluarse con respecto a si infieren o no en la seguridad de la prueba. A continuación mostramos un código en el que variamos tanto la cantidad de columnas que se utilizan, como el algoritmo que se usa para calcular la traza. Con estos, imprimimos los valores del tiempo de generación de la prueba y verificación, y el tamaño en memoria de la prueba. Se puede observar como ambos disminuyen con respecto a los parámetros comentados. 


In [75]:
let trace_length = 1024;
let power_2_tracelen = 10;
let max_power_2_jump_len = 4;
let proof_options = ProofOptions::default_test_options();
fn update_public_inputs(
    jump_len: u32,
    original_pub_inputs: PellPublicInputs<F>,
    trace: TraceTable<F>
) -> PellPublicInputs<F> {
    if jump_len == 1 {
        return original_pub_inputs;
    }
    PellPublicInputs {
        a0: trace.get((jump_len - 1) as usize, 0),
        a1: trace.get((2 * jump_len - 1) as usize, 0),
    }
}
let n_loops = 30;
for jump_factor in 0..max_power_2_jump_len {
    let jump_len = u32::pow(2,jump_factor);
    // cambia la ecuacion de recurrencia al ser una subsecuencia.
    let (c0, c1) = update_factors(jump_len);
    let pub_inputs = update_public_inputs(jump_len, original_pub_inputs.clone(), trace.clone());
    println!("\n| #Jumps | #Cols | Largo Traza | Ej. Traza | Ej. Prueba | Ej. Verificacion | Tamaño prueba |");
    for col_factor in 0..power_2_tracelen-jump_factor-1 {
        let n_cols = change_n_cols(usize::pow(2,col_factor));
        let trace_length = usize::pow(2,power_2_tracelen - col_factor - jump_factor);
        let (trace_time, proof_time, verify_time, proof_size) = 
                     run_many_test_prove(n_cols, trace_length, 
                                         c0.clone(), c1.clone(),
                                         pub_inputs.clone(), proof_options.clone(), 
                                         expected_result.clone(), n_loops);
        println!("|{:>7} |{:>5}  | {:>7}     |{:>6}µs   |{:>6}ms    | {:>8}ms       | {:>9}b    |",
                  jump_len, n_cols, trace_length, trace_time.as_micros(), 
                  proof_time.as_millis(), verify_time.as_millis(), proof_size);
    }
    println!();
}

La restriccion de transicion es: P(n) = 2 * P(n-1) + (1) * P(n-2)

| #Jumps | #Cols | Largo Traza | Ej. Traza | Ej. Prueba | Ej. Verificacion | Tamaño prueba |
|      1 |    1  |    1024     |    64µs   |    36ms    |        2ms       |     20344b    |
|      1 |    2  |     512     |    64µs   |    18ms    |        1ms       |     17704b    |
|      1 |    4  |     256     |    63µs   |    10ms    |        0ms       |     15448b    |
|      1 |    8  |     128     |    70µs   |     6ms    |        0ms       |     13704b    |
|      1 |   16  |      64     |    69µs   |     4ms    |        0ms       |     12792b    |
|      1 |   32  |      32     |    73µs   |     3ms    |        0ms       |     13352b    |
|      1 |   64  |      16     |    82µs   |     3ms    |        0ms       |     16664b    |
|      1 |  128  |       8     |    85µs   |     4ms    |        0ms       |     25288b    |
|      1 |  256  |       4     |    86µs   |     5ms    |        1ms       |     44344b    |

La

()